In [1]:
# 경고 필터링
import warnings
warnings.filterwarnings('ignore')

import os, sys, gc, warnings, random, collections

import datetime
import dateutil.relativedelta

# 데이터 조작
import pandas as pd 
import numpy as np

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 머신러닝
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, roc_curve
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold, GroupKFold
from sklearn.ensemble import RandomForestClassifier

import lightgbm as lgb

from tqdm.notebook import trange, tqdm

from IPython.display import display

%matplotlib inline

pd.options.display.max_rows = 10000
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = 1000

In [2]:
df = pd.read_csv('../../input/train.csv', parse_dates=['order_date'])
df['year_month'] = df.order_date.dt.strftime('%Y-%m')
df.head()

,order_id,product_id,description,quantity,order_date,price,customer_id,country,total,year_month
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,11.4675,13085,United Kingdom,137.61,2009-12
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,11.1375,13085,United Kingdom,133.65,2009-12
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,11.1375,13085,United Kingdom,133.65,2009-12
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,3.4650,13085,United Kingdom,166.32,2009-12
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,2.0625,13085,United Kingdom,49.50,2009-12


In [9]:
target = datetime.datetime.strptime('2011-12', "%Y-%m")
prev = target - dateutil.relativedelta.relativedelta(years=1)
pprev = prev - dateutil.relativedelta.relativedelta(years=1)
prev, pprev = prev.strftime('%Y-%m'), pprev.strftime('%Y-%m')
prev, pprev

('2010-12', '2009-12')

In [10]:
groupby = df.groupby(['customer_id', 'year_month'])['total'].sum()
groupby = groupby.unstack()

In [11]:
groupby.loc[:, [prev, pprev]]

year_month,2010-12,2009-12
customer_id,,
12346,NaN,1.872750e+02
12347,1174.4535,NaN
12348,1473.1200,NaN
12349,NaN,-3.984750e+01
12350,NaN,NaN
12351,NaN,NaN
12352,NaN,NaN
12353,NaN,NaN
12354,NaN,NaN
